<a href="https://colab.research.google.com/github/PsorTheDoctor/Sekcja-SI/blob/master/neural_networks/RNN/performance_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perfonmance RNN: Generowanie muzyki na LSTM
## Ustawienia środowiska

In [0]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 build-essential libasound2-dev libjack-dev
!pip install -U magenta pyfluidsynth

In [0]:
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

In [4]:
!gsutil -m cp gs://download.magenta.tensorflow.org/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /tmp/

Copying gs://download.magenta.tensorflow.org/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2...
\ [1/1 files][591.9 MiB/591.9 MiB] 100% Done  23.2 MiB/s ETA 00:00:00           
Operation completed over 1 objects/591.9 MiB.                                    


In [0]:
import os
from magenta.models.performance_rnn import performance_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from magenta.music.protobuf import generator_pb2
from magenta.music.protobuf import music_pb2

import magenta.music as mm

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

BUNDLE_DIR = '/tmp/'
MODEL_NAME = 'performance_with_dynamics'
BUNDLE_NAME = MODEL_NAME + '.mag'

In [9]:
mm.notebook_utils.download_bundle(BUNDLE_NAME, BUNDLE_DIR)

## Generowanie sekwencji

In [16]:
bundle = sequence_generator_bundle.read_bundle_file(os.path.join(BUNDLE_DIR, BUNDLE_NAME))
generator_map = performance_sequence_generator.get_generator_map()
generator = generator_map[MODEL_NAME](checkpoint=None, bundle=bundle)
generator.initialize()
generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = 1  # Im większa tym bardziej losowa
generate_section = generator_options.generate_sections.add(start_time=0, end_time=30)
sequence = generator.generate(music_pb2.NoteSequence(), generator_options)

mm.plot_sequence(sequence)

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /tmp/tmp0gouehrc/model.ckpt
INFO:tensorflow:Need to generate 2899 more steps for this sequence, will try asking for 1740 RNN steps
INFO:tensorflow:Beam search yields sequence with log-likelihood: -4301.637695 


In [17]:
mm.play_sequence(sequence, mm.midi_synth.fluidsynth,
                 sf2_path='/tmp/Yamaha-C5-Salamander-JNv5.1.sf2')